In [ ]:
!pip install psycopg2-binary

In [12]:
import psycopg2

In [13]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")

In [14]:
db = conn.cursor()
#query point : 105.8475554, 21.007130000404484
#test point :105.85062869999999 21.00922750040452

# IER

In [15]:
#ST_AsText(ST_Union(geom_way)) return multilineString
def IED(long, lat):
    path=[]
    #find euclide NN
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                limit 1""")
    qPoint=db.fetchall()
    #find network routing
    #qPoint[0][0],qPoint[0][1]: longtitude and latitude of NN point 
    db.execute(f"""
                WITH start AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                      LIMIT 1
                    ),
                destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({qPoint[0][0]} {qPoint[0][1]})'),
                      4326)
                  LIMIT 1
                    )
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    DEmax= result[0][0] #network routing distance from NN point to given point
    path= result[0][1]
    #find candidate in this network routing
    
    #DE max: network routing from given point to NN (NN by euclid)
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<=  {DEmax}
                """)
    NN=db.fetchall()
    #find result
    for i in NN:
        db.execute(f"""
                WITH start AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long} {lat})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT (ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][1]>DEmax) : continue
        else:
            path=x[0][0]
            DEmax =x[0][1]
            qPoint=i
    return qPoint, DEmax, path,NN

In [12]:
IED(105.8042238, 21.030628300404896)

((105.8068682, 21.033045300404943),
 471.871298337968,
 '0105000020E6100000050000000102000000100000003FE834666D735A40120A5BFD2208354045189A466E735A40A873452921083540380884E76F735A402CA688C21D08354017050B7176735A40759DFCCC0F08354039A407F478735A403B2F5FA80D083540BC3A6CC779735A409BB3999E0B0835402672C1197C735A4090DD054A0A083540D29B40C77D735A403B7716180D0835403C1B4D897F735A4043C5387F1308354059CFFF6181735A40FFBDCA9019083540B78CE5A782735A409E0546031D0835400A5A924D83735A401B272AC01C08354026080DB386735A40C2572A5E1B083540F60A0BEE87735A40B1EC38341B083540B3F803D48A735A40E62D0DB21B0835406F675F7990735A40ED4D67823C0835400102000000020000006F675F7990735A40ED4D67823C08354008D8FD1994735A4043BB54B65908354001020000000400000008D8FD1994735A4043BB54B6590835408421BC8795735A40B052E68B62083540F4EAD27597735A401C29B6DD5F083540FA9001EA97735A40F832F6CA61083540010200000002000000FA9001EA97735A40F832F6CA610835404C135BE699735A40D7F6764B720835400102000000030000004C135BE699735A40D7F6764B720835403F17B25F9C735A40D8AC44EA870

In [38]:
import timeit
#index
elapsed_time = timeit.timeit("IED(105.8042238, 21.030628300404896)", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 303.0264481999993 seconds


In [16]:
#non index
elapsed_time = timeit.timeit("IED(105.8042238, 21.030628300404896)", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 404.2515760000001 seconds


# NN to 2 given point

## method 1 candidat by IER

In [23]:
# find NN by IER
def IER(long, lat):
    path=[]
    #find euclide NN
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
    qPoint=db.fetchall()
    #find network routing
    db.execute(f"""
                WITH start AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                      LIMIT 1
                    ),
                destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({qPoint[0][0]} {qPoint[0][1]})'),
                      4326)
                  LIMIT 1
                    )
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    DEmax= result[0][0]
    path= result[0][1]
    #find candidate in this network routing
    db.execute(f"""select long, lat FROM geometries
                where amenity='restaurant'
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<=  {DEmax}
                """)
    NN=db.fetchall()
    #find result
    for i in NN:
        db.execute(f"""
                WITH start AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long} {lat})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT (ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][1]>DEmax) : continue
        else:
            path=x[0][0]
            DEmax =x[0][1]
            qPoint=i
    return qPoint, DEmax, path,NN

In [34]:
#POINT1(105.80145149999998 21.035779800404985)
#POINT2(105.78767219999999 21.036938200405007)

In [35]:
IER(105.80145149999998, 21.035779800404985)
##NN1=

((105.8000055, 21.03867230040504),
 606.024513451644,
 '0105000020E61000000A000000010200000002000000A6D590B847735A40D7E2AEB9FE0835403A7BC26D48735A403C6068BFFF083540010200000002000000AD4214DD44735A4021730A97FA083540A6D590B847735A40D7E2AEB9FE083540010200000002000000DE42B2DB42735A400466CFAFF7083540AD4214DD44735A4021730A97FA083540010200000003000000DE42B2DB42735A400466CFAFF7083540F673AF3841735A40C6E5D3BE14093540F1434A5840735A4091FEA89729093540010200000002000000C029070E43735A403EC10F1835093540F1434A5840735A4091FEA89729093540010200000004000000C679EEF34E735A409D514E0F65093540167431BC48735A40F14927124C093540ED56E07547735A40D057DAF346093540C029070E43735A403EC10F1835093540010200000003000000A7CA9CD351735A408AE365F272093540C52BFF1051735A40383932456F093540C679EEF34E735A409D514E0F650935400102000000020000004D37E4FA52735A4075A84B6B78093540A7CA9CD351735A408AE365F2720935400102000000030000004D37E4FA52735A4075A84B6B780935409B4D918D51735A4043E4F4F57C09354032BA72AC41735A404DB21F73AF09354001020000000200000032

In [36]:
IER(105.78767219999999, 21.036938200405007)
#NN2=

((105.78914509999998, 21.035509500404984),
 276.80359196156576,
 '0105000020E610000004000000010200000002000000434EA95C72725A409D34684359093540F4E967A068725A4001C68E215B0935400102000000050000000770C4A489725A4088F0D4C853093540C7BAB88D86725A40527BB6B354093540C8050C3785725A40ABDA24F5540935403E1350977B725A40DF9FF76B57093540434EA95C72725A409D346843590935400102000000020000000770C4A489725A4088F0D4C85309354079628CFE86725A40A90DF38A4C09354001020000000300000079628CFE86725A40A90DF38A4C093540329FBD4486725A40D584A39A4809354098C68A7585725A407517CD6F3D093540',
 [(105.78914509999998, 21.035509500404984)])

In [24]:
#calculating the cost by network routing from NN point(follow by IER) to 2 given point
def cost2point(long, lat, long1, lat1, long2, lat2):
    db.execute(f"""
                WITH start1 AS (
              SELECT topo.source 
              FROM osm_2po_4pgr as topo
              ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long1} {lat1})'),
                      4326)
                      LIMIT 1
                    ),
                start2 AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long2} {lat2})'),
                      4326)
                  LIMIT 1
                    ),
                    destination AS (
                  SELECT topo.source 
                  FROM osm_2po_4pgr as topo
                  ORDER BY topo.geom_way <-> ST_SetSRID(
                ST_GeomFromText('POINT ({long} {lat})'),
                      4326)
                  LIMIT 1
                    )   
                SELECT sum(di.cost) as realcost,ST_Union(geom_way) as path
                    FROM pgr_dijkstra('
                SELECT id,
                     source,
                     target,
                     ST_Length(ST_Transform(geom_way, 3857)) AS cost
                        FROM osm_2po_4pgr',
                    array(SELECT source FROM start1 UNION SELECT source from start2),
                    array(SELECT source FROM destination),
                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                      ON   di.edge = pt.id
                      """)
    result=db.fetchall()
    cost= result[0][0]
    return cost

In [40]:
cost2point(105.8000055, 21.03867230040504,105.80145149999998 ,21.035779800404985,105.78767219999999, 21.036938200405007)
#sum cost of NN1

2262.4613884916093

In [42]:
cost2point(105.78914509999998, 21.035509500404984,105.80145149999998, 21.035779800404985,105.78767219999999, 21.036938200405007)
#sum_cost NN2

1813.2061564787077

In [43]:
#cost by NN1 is smaller, then => query point from point2 with euclide = NN1, 105.84089229999998, 20.995627400404274

In [25]:
from functools import reduce

In [28]:
#finding all candidate which is within smaller network routing distance

def candidate(long1,lat1,long2,lat2,max1):
    waste=0
    path=1
    result=1
    end=[]
    db.execute(f"""
    select long, lat FROM geometries
                where amenity='restaurant'
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long2}, {lat2}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    qPoint1=db.fetchall()
    db.execute(f"""
    select long, lat FROM geometries
                where amenity='restaurant'
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long1}, {lat1}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    qPoint2=db.fetchall()
    intersection = reduce(lambda acc, x: acc + [x] if x in qPoint1 and x not in acc else acc, qPoint2, [])
    union = set(qPoint1).union(qPoint2)

    #find network routing
    for i in intersection:
        db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT {i} as point ,(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        end.append(x)
    sorted_x= sorted(end, key=lambda x: x[0][-1])
    return sorted_x[0] #result point, path, cost
        
#         #end.append(x[0][1])
#         if(x[0][1]>max1) : continue
#         else:
#             path=x[0][0]
#             waste =x[0][1]
#             result=i
#     return result, waste, path

    #return end

#### My function

In [29]:
def path2point(long1, lat1, long2,lat2):
    NN1=IER(long1,lat1)
    NN2=IER(long2,lat2)
    x1=cost2point(NN1[0][0],NN1[0][1],long1, lat1, long2,lat2)
    x2=cost2point(NN2[0][0],NN2[0][1],long1, lat1, long2,lat2)
    if x1>=x2:
        result=candidate(long1, lat1, long2,lat2,x2)
    else: result=candidate(long1, lat1, long2,lat2,x1)
    return result

In [62]:
path2point(105.80145149999998 ,21.035779800404985,105.78767219999999, 21.036938200405007)

[('(105.79514919999998,21.03487360040497)',
  '0105000020E61000001E000000010200000002000000A6D590B847735A40D7E2AEB9FE0835403A7BC26D48735A403C6068BFFF083540010200000002000000AD4214DD44735A4021730A97FA083540A6D590B847735A40D7E2AEB9FE083540010200000002000000DE42B2DB42735A400466CFAFF7083540AD4214DD44735A4021730A97FA0835400102000000030000006631B1F938735A40E15E99B7EA08354087DEE2E13D735A4049FDAB7DF0083540DE42B2DB42735A400466CFAFF7083540010200000002000000AE5978A837735A404DA5FA29E90835406631B1F938735A40E15E99B7EA0835400102000000020000000A16E2EC31735A403485CE6BEC083540AE5978A837735A404DA5FA29E90835400102000000030000000A16E2EC31735A403485CE6BEC08354034F7EBA930735A4084961A46E608354051EADCFE26735A40559EE51EB708354001020000000200000051EADCFE26735A40559EE51EB708354088AA4EBD24735A40DE1C531CAC08354001020000000200000088AA4EBD24735A40DE1C531CAC083540EF3614E321735A401135D1E7A3083540010200000002000000EF3614E321735A401135D1E7A30835408109DCBA1B735A4081762C5B900835400102000000040000008109DCBA1B735A4081762C5B9

In [39]:
#index
elapsed_time = timeit.timeit("path2point(105.80145149999998 ,21.035779800404985,105.78767219999999, 21.036938200405007)", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 765.8553222 seconds


In [30]:
#non index
elapsed_time = timeit.timeit("path2point(105.80145149999998 ,21.035779800404985,105.78767219999999, 21.036938200405007)", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 1275.3872347000001 seconds


## method 2:candidat by euclide

In [ ]:
from functools import reduce

In [31]:
#AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))

def euclid(long,lat):
    db.execute(f"""select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries
                where amenity='restaurant'
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
    NN=db.fetchall()
    return NN

In [14]:
#POINT1(105.80145149999998 21.035779800404985)
x1=euclid(105.80145149999998, 21.035779800404985) #given point 1
x1 #NN của point1

[(105.8000055, 21.03867230040504, 380.69136656644343)]

In [15]:
#POINT2(105.78767219999999 21.036938200405007)
x2=euclid(105.78767219999999, 21.036938200405007) #given point 2
x2 #NN của point2

[(105.78914509999998, 21.035509500404984, 236.47278118702116)]

In [32]:
#AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE p.osm_id ='-9421134'))
def find_NN(long,lat,max1):   
    db.execute(f"""
    select long, lat FROM geometries
                where amenity='restaurant'
                AND  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({long}, {lat}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) <=  {max1}
                                
                """)
    Point=db.fetchall()
    return Point

In [19]:
NN1=find_NN(105.80145149999998, 21.035779800404985,x1[0][2])  #x1[0][2] = max1
NN1

[(105.8000055, 21.03867230040504)]

In [21]:
NN2=find_NN(105.78767219999999, 21.036938200405007,x1[0][2])
NN2

[(105.7857867, 21.038758100405033),
 (105.7855911, 21.03890420040504),
 (105.78562479999998, 21.03890360040504),
 (105.78545609999999, 21.038906700405043),
 (105.78548989999999, 21.03890600040504),
 (105.7855236, 21.03890550040504),
 (105.7854859, 21.038763600405037),
 (105.7889278, 21.034573200404967),
 (105.78894929999998, 21.034648300404964),
 (105.78900139999999, 21.034937500404972),
 (105.78914509999998, 21.035509500404984),
 (105.78967349999999, 21.034375400404958),
 (105.7893302, 21.034508100404963)]

In [22]:
intersection = reduce(lambda acc, x: acc + [x] if x in NN1 and x not in acc else acc, NN2, [])
intersection

[]

In [23]:
union=set(NN1).union(NN2)
union

{(105.78545609999999, 21.038906700405043),
 (105.7854859, 21.038763600405037),
 (105.78548989999999, 21.03890600040504),
 (105.7855236, 21.03890550040504),
 (105.7855911, 21.03890420040504),
 (105.78562479999998, 21.03890360040504),
 (105.7857867, 21.038758100405033),
 (105.7889278, 21.034573200404967),
 (105.78894929999998, 21.034648300404964),
 (105.78900139999999, 21.034937500404972),
 (105.78914509999998, 21.035509500404984),
 (105.7893302, 21.034508100404963),
 (105.78967349999999, 21.034375400404958),
 (105.8000055, 21.03867230040504)}

In [20]:
long1=105.8367243
lat1= 20.991411700404193
long2=105.8502594
lat2=20.997397700404303
result=[]
db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({intersection[0][0]} {intersection[0][1]})'),4326)
                          LIMIT 1
                                    )
                SELECT (ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    array(SELECT source FROM destination),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
x=db.fetchall()
if(x[0][-1] is not None):       
    result.append(x)
            
ending_result= sorted(result, key=lambda x: x[0][-1])[0]

In [58]:
a=set(NN1).union(NN2)
for i in a:
    print(i, i[0],i[1])

(105.85063121852589, 20.999756697279977) 105.85063121852589 20.999756697279977
(105.84333377184484, 21.002040406245815) 105.84333377184484 21.002040406245815
(105.84408432517345, 20.997963931988973) 105.84408432517345 20.997963931988973
(105.84415592757635, 20.99778753353857) 105.84415592757635 20.99778753353857


In [43]:
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")
db = conn.cursor()

In [33]:
#find best result in candidate set
def problem9(long1, lat1, long2, lat2):
    x1= euclid(long1,lat1)
    x2= euclid(long2, lat2)
    result=[]
    if x1[0][2] > x2[0][2]:
        max_e=x1[0][2]
    else:
        max_e=x2[0][2]
    NN1= find_NN(long1,lat1, max_e)
    NN2= find_NN(long2, lat2,max_e)
    intersection = reduce(lambda acc, x: acc + [x] if x in NN1 and x not in acc else acc, NN2, [])
    union = set(NN1).union(NN2)
    result=[]
    for i in union: # or intersection
        db.execute(f"""
                WITH start1 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long1} {lat1})'),4326)
                      LIMIT 1
                                ),
                    start2 AS (
                      SELECT topo.source 
                      FROM osm_2po_4pgr as topo
                      ORDER BY topo.geom_way <-> ST_SetSRID(
                                ST_GeomFromText('POINT ({long2} {lat2})'),4326)
                      LIMIT 1
                                ),
                    destination AS (
                          SELECT topo.source 
                          FROM osm_2po_4pgr as topo
                          ORDER BY topo.geom_way <-> ST_SetSRID(
                            ST_GeomFromText('POINT ({i[0]} {i[1]})'),4326)
                          LIMIT 1
                                    )
                SELECT  {i} as point,(ST_Union(geom_way)), sum(di.cost) as realcost
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM destination),
                                    array(SELECT source FROM start1 UNION SELECT source FROM start2),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
        x=db.fetchall()
        if(x[0][-1] is not None):       
            result.append(x)
            
    sorted_x= sorted(result, key=lambda x: x[0][-1])
    return sorted_x[0] #result point, path, cost

### my function

In [25]:
#POINT1(105.80145149999998 21.035779800404985)
#POINT2(105.78767219999999 21.036938200405007)
result=problem9(long1=105.80145149999998,
lat1= 21.035779800404985,
long2=105.78767219999999,
lat2=21.036938200405007)

In [26]:
result  #union

[('(105.78894929999998,21.034648300404964)',
  '0105000020E61000002100000001020000000200000079628CFE86725A40A90DF38A4C093540329FBD4486725A40D584A39A48093540010200000002000000329FBD4486725A40D584A39A4809354098C68A7585725A407517CD6F3D0935400102000000020000000770C4A489725A4088F0D4C85309354079628CFE86725A40A90DF38A4C093540010200000002000000D60DCAEA8C725A409A33EC41520935400770C4A489725A4088F0D4C8530935400102000000050000007C105DAB98725A40A6D82CF246093540B9EF62AB95725A40578B998E4A0935400F42E50492725A40EA245B5D4E093540ECA2E8818F725A4012842BA050093540D60DCAEA8C725A409A33EC41520935400102000000020000004C3ED1D09A725A40BFB0A140440935407C105DAB98725A40A6D82CF24609354001020000000300000098C11891A8725A402DD21FF53209354004087A03A7725A402C561EEE340935404C3ED1D09A725A40BFB0A14044093540010200000003000000314C5C7DAB725A403BD400EF2E093540C123850DAA725A4064D7ACE93009354098C11891A8725A402DD21FF5320935400102000000020000007D2E0906C6725A40E2D528DA0B093540314C5C7DAB725A403BD400EF2E09354001020000000300000046F5317AC9

In [40]:
#index
elapsed_time = timeit.timeit("""problem9(long1=105.80145149999998,
lat1= 21.035779800404985,
long2=105.78767219999999,
lat2=21.036938200405007)""", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 253.18822329999966 seconds


In [34]:
#no index
elapsed_time = timeit.timeit("""problem9(long1=105.80145149999998,
lat1= 21.035779800404985,
long2=105.78767219999999,
lat2=21.036938200405007)""", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 406.2500336000003 seconds


# with a set of point

In [70]:
import psycopg2
conn = psycopg2.connect(database="osm",
                        host="localhost",
                        user="postgres",
                        password="00000",
                        port="5432")
db = conn.cursor()

In [7]:
db.execute(f"""
    select long, lat from geometries_vn
    where amenity ='hospital'
    AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE name = 'Quận Hoàn Kiếm'))
""")
h=db.fetchall()

In [35]:
import itertools
# for b in h:
#     print(b)

### my function

In [36]:
dis=0
end=[]
candidat=[]

def bePoint():
    db.execute(f"""
    select long, lat from geometries
    where amenity ='hospital'
    AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE osm_id='-9421131'))   
""")
    h=db.fetchall() #candidat of Hospital
    for b in h:
        b=b
        db.execute(f"""
    select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(105.8548261, 21.024432000404786), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries
                where amenity='restaurant'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(105.8548261, 21.024432000404786), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                limit 1
    
    """)
    
        r= db.fetchall()  #candidat of Restaurant
#     res.append(x)
#     sorted_res = sorted(res, key=lambda x: x[0][-1])
    #r[0][0], r[0][1]: long lat of each Restaurant point
        db.execute(f"""select long, lat,
    ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
    FROM geometries
                where amenity='atm'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                order by ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857)) 
                                
                limit 1""")
        a= db.fetchall()   #candidat of ATM 
    
        if r[0][2]> a[0][2]:  # r[0][2]: distance from Restaurant to Hospital
            dis= r[0][2]
        else:
            dis= a[0][2]   ## a[0][2]: distance from Restaurant to ATM
        
        db.execute(f"""
    select long, lat
    FROM geometries
                where amenity='hospital'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<={dis}  
    """)
        h_nn= db.fetchall() #new candidat of Hospital
    
        db.execute(f"""
    select long, lat
    FROM geometries
                where amenity='atm'
                AND ST_Within(way, (SELECT way FROM planet_osm_polygon p WHERE  osm_id='-9421131'))
                and ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({r[0][0]}, {r[0][1]}), 4326), 3857), 
                                ST_Transform(ST_SetSRID(ST_MakePoint(long, lat), 4326), 3857))<={dis}  
    """)
        a_nn= db.fetchall() #new candidat of ATM
    
        for i, z in itertools.product(h_nn,a_nn):
            db.execute(f"""
            WITH start1 AS (
          SELECT topo.source
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
        ST_GeomFromText('POINT({r[0][0]} {r[0][1]})'),4326) 
          LIMIT 1)

        ,start2 as (
        SELECT topo.source
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
            ST_GeomFromText('POINT({z[0]} {z[1]})'),4326) 
          LIMIT 1)
        ,destination AS (
          SELECT topo.source 
          FROM osm_2po_4pgr as topo
          ORDER BY topo.geom_way <-> ST_SetSRID(
        ST_GeomFromText('POINT({i[0]} {i[1]})'),4326) 
          LIMIT 1
        )
        SELECT ST_Union(geom_way), sum(di.cost) as realcost, {i}, {z},{r[0]}
                    FROM pgr_dijkstra('
                            SELECT id,
                                   source,
                                   target,
                                    ST_Length(ST_Transform(geom_way, 3857)) AS cost
                            FROM osm_2po_4pgr',
                                    array(SELECT source FROM start1 ),
                                    array(SELECT source FROM destination UNION SELECT source FROM start2),
                                    directed := false) AS di
                    JOIN   osm_2po_4pgr AS pt
                    ON   di.edge = pt.id
                      """)
            c= db.fetchall() #candidate of 3 point and cost    
#         Some point has not path and cost, it means they are not reach to others
#         if c[0][0] is not None:  
            candidat.append(c)
        end= sorted(candidat, key=lambda x: x[0][1])[0] #result for the smallest cost
# #     candidat1= sorted_c
#     end.append(candidat)
    return end

In [77]:
bePoint() #path, cost, hospital, atm, restaurant

[('0102000020E61000000400000004BBAB68BD765A40126E8D637F0635406A6CAF05BD765A40BBAAFDE77A063540CA07F30DBB765A40A314BE0864063540BE45CC91BA765A400BF20E4B5E063540',
  63.24046506662777,
  '(105.8548261,21.024432000404786)',
  '(105.8553356,21.024990100404796)',
  '(105.8549087,21.024559900404793,17.810343901958767)')]

In [41]:
#index
elapsed_time = timeit.timeit("bePoint()", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 49.919413100000384 seconds


In [37]:
#no index
elapsed_time = timeit.timeit("bePoint()", globals=globals(), number=1)
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 72.12470289999965 seconds
